# Data Exploration
## Local machine

In [ ]:
import pandas as pd
df = pd.read_csv('../data/day.csv')
df['date'] = df['dteday'].astype('datetime64[s]')
df.plot('date',['cnt', ])


In [ ]:
df.plot('date', 'temp')
df.plot( 'temp', 'cnt', kind='scatter')

In [ ]:
df[df.yr==0].plot( 'temp', 'cnt', kind='scatter')
df[df.yr==1].plot( 'temp', 'cnt', kind='scatter')


In [ ]:
df['temp2'] = (df['temp']-0.6)**2
df[df.yr==1].plot( 'temp2', 'cnt', kind='scatter')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# load data

def get_data():
    df = pd.read_csv('../data/hour.csv')
    X = df[['hr', 'temp', 'hum', 'windspeed','workingday', 'weathersit' ]]
    X['hr_sin'] =  X['hr'].apply(lambda x: np.sin(2 * np.pi * x / 24))
    X['hr_cos'] =  X['hr'].apply(lambda x: np.cos(2 * np.pi * x / 24))
    X['temp2'] = (X['temp'] - 0.6)**2
    y = df['cnt']
    return X, y

def encode_cols(X, cols=[]):
    if len(cols)==0:
        return X
    encoder = OneHotEncoder(sparse=False)
    X_encoded = encoder.fit_transform(X[cols])
    X.drop(cols, axis=1, inplace=True)
    X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(cols))
    Xe = pd.concat([X, X_encoded], axis=1)
    return Xe

def solve(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', rmse)

    # plot the predicted vs. actual values
    fig , ax = plt.subplots(1)
    ax.scatter(y_test, y_pred,s=2, alpha=0.5)
    ax.set_xlabel('Actual Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title('Predicted vs. Actual Values')
    fig.show()


In [ ]:

models = [
    LinearRegression(fit_intercept=False), 
    RandomForestRegressor(n_estimators=10)
]

for m in models:
    X,y = get_data()
    Xe = encode_cols(X, ['workingday', 'weathersit'])
    solve (m, Xe, y)

In [ ]:
LinearRegression 132
RandomForest(100) 76
RandomForest(10) 79